In [4]:
import com.toldoven.aoc.notebook.AocClient

val aoc = AocClient.fromEnv().interactiveDay(2018, 11)

aoc.viewPartOne()

--- Day 11: Chronal Charge --- 
 You watch the Elves and their sleigh fade into the distance as they head toward the North Pole. 
 Actually, you're the one fading. The falling sensation returns. 
 The low fuel warning light is illuminated on your wrist-mounted device. Tapping it once causes it to project a hologram of the situation: a 300x300 grid of fuel cells and their current power levels, some negative. You're not sure what negative power means in the context of time travel, but it can't be good. 
 Each fuel cell has a coordinate ranging from 1 to 300 in both the X (horizontal) and Y (vertical) direction. In X,Y notation, the top-left cell is 1,1 , and the top-right cell is 300,1 . 
 The interface lets you select any 3x3 square of fuel cells. To increase your chances of getting to your destination, you decide to choose the 3x3 square with the largest total power . 
 The power level in a given fuel cell can be found through the following process: 
 
 Find the fuel cell's rack ID , which is its X coordinate plus 10 . 
 Begin with a power level of the rack ID times the Y coordinate . 
 Increase the power level by the value of the grid serial number (your puzzle input). 
 Set the power level to itself multiplied by the rack ID . 
 Keep only the hundreds digit of the power level (so 12 3 45 becomes 3 ; numbers with no hundreds digit become 0 ). 
 Subtract 5 from the power level. 
 
 For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8 : 
 
 The rack ID is 3 + 10 = 13 . 
 The power level starts at 13 * 5 = 65 . 
 Adding the serial number produces 65 + 8 = 73 . 
 Multiplying by the rack ID produces 73 * 13 = 949 . 
 The hundreds digit of 9 49 is 9 . 
 Subtracting 5 produces 9 - 5 = 4 . 
 
 So, the power level of this fuel cell is 4 . 
 Here are some more example power levels: 
 
 Fuel cell at   122,79 , grid serial number 57 : power level -5 . 
 Fuel cell at 217,196 , grid serial number 39 : power level   0 . 
 Fuel cell at 101,153 , grid serial number 71 : power level   4 . 
 
 Your goal is to find the 3x3 square which has the largest total power. The square must be entirely within the 300x300 grid. Identify this square using the X,Y coordinate of its top-left fuel cell . For example: 
 For grid serial number 18 , the largest total 3x3 square has a top-left corner of 33,45 (with a total power of 29 ); these fuel cells appear in the middle of this 5x5 region: 
 -2 -4 4 4 4
-4 4 4 4 -5
 4 3 3 4 -4
 1 1 2 4 -3
-1 0 2 -5 -2
 
 For grid serial number 42 , the largest 3x3 square's top-left is 21,61 (with a total power of 30 ); they are in the middle of this region: 
 -3 4 2 2 2
-4 4 3 3 4
-5 3 3 4 -4
 4 3 3 4 -3
 3 3 3 -5 -1
 
 What is the X,Y coordinate of the top-left fuel cell of the 3x3 square with the largest total power?

In [5]:
val input = aoc.input()
input

[9435]

In [40]:
/*
 For example, to find the power level of the fuel cell at 3,5 in a grid with serial number 8:

The rack ID is 3 + 10 = 13.
The power level starts at 13 * 5 = 65.
Adding the serial number produces 65 + 8 = 73.
Multiplying by the rack ID produces 73 * 13 = 949.
The hundreds digit of 949 is 9.
Subtracting 5 produces 9 - 5 = 4.

The power level in a given fuel cell can be found through the following process:

Find the fuel cell's rack ID, which is its X coordinate plus 10.
Begin with a power level of the rack ID times the Y coordinate.
Increase the power level by the value of the grid serial number (your puzzle input).
Set the power level to itself multiplied by the rack ID.
Keep only the hundreds digit of the power level (so 12345 becomes 3; numbers with no hundreds digit become 0).
Subtract 5 from the power level.
**/

fun Point.powerLevel(serialNumber: Int): Long {
    val rackid = this.x.toLong() + 10
    var powerLevel = rackid * this.y
    powerLevel += serialNumber
    powerLevel *= rackid
    val hundredsDigit = powerLevel / 100 % 10
    return hundredsDigit - 5
}

//val serialNumber = input[0].toInt()
val serialNumber = 18

val points = (1..300)
    .flatMap { x -> (1..300).map { y -> Point(x, y) } }
    .toSet()

val cache = mutableMapOf<Pair<Point, Int>, Long>()

fun Point.powerLevelNeighbourhood(points: Set<Point>, size: Int): Long {
    if (size == 1) {
        this.powerLevel(serialNumber).run {
            cache[Pair(this@powerLevelNeighbourhood, size)] = this
            return this
        }
    }
    cache[Pair(this, size)]?.run {
        return this
    }
    val previousPowerLevel = this.powerLevelNeighbourhood(points, size - 1)

    val rowPoints = (0 until size).map { Point(this.x + size - 1, this.y + it) }
    val colPoints = (0 until size).map { Point(this.x + it, this.y + size - 1) }
    val allPoints = (rowPoints + colPoints).toSet()
    //println(allPoints)

    if (allPoints.any { it !in points }) {
        cache[Pair(this, size)] = Long.MIN_VALUE
        return Long.MIN_VALUE
    }

    val toReturn = allPoints
        .sumOf { it.powerLevel(serialNumber) } + previousPowerLevel
    cache[Pair(this, size)] = toReturn
    return toReturn
}

val max = points.flatMap { point -> (1..300).map { s -> Pair(point, s) } }
    .maxBy { (point, s) -> point.powerLevelNeighbourhood(points, s) }

max


org.jetbrains.kotlinx.jupyter.exceptions.ReplInterruptedException: The execution was interrupted

In [9]:
aoc.submitPartOne("20,41")

Your answer: 20,41. 
 That's the right answer! You are one gold star closer to fixing the time stream. [Continue to Part Two]

In [10]:
aoc.viewPartTwo()

--- Part Two --- 
 You discover a dial on the side of the device; it seems to let you select a square of any size , not just 3x3. Sizes from 1x1 to 300x300 are supported. 
 Realizing this, you now must find the square of any size with the largest total power . Identify this square by including its size as a third parameter after the top-left coordinate: a 9x9 square with a top-left corner of 3,5 is identified as 3,5,9 . 
 For example: 
 
 For grid serial number 18 , the largest total square (with a total power of 113 ) is 16x16 and has a top-left corner of 90,269 , so its identifier is 90,269,16 . 
 For grid serial number 42 , the largest total square (with a total power of 119 ) is 12x12 and has a top-left corner of 232,251 , so its identifier is 232,251,12 . 
 
 What is the X,Y,size identifier of the square with the largest total power?